## ℹ️ Info

* **forked original great work kernels**
    * https://www.kaggle.com/code/siddhvr/aes-2-0-deberta-lgbm-baseline
    * https://www.kaggle.com/code/olyatsimboy/5-fold-deberta-lgbm
    * https://www.kaggle.com/code/aikhmelnytskyy/quick-start-lgbm
    

* **2024/04/15 My Additional**
    * add CountVectorizer FEs.
    ```
    vectorizer_cnt = CountVectorizer(
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(2,3),
            min_df=0.10,
            max_df=0.85,
    ```

* **2024/04/21 My Additional**
    * Add MetaFEs. Train deberta-v3-large local(5Fold SKF)
    * https://www.kaggle.com/datasets/hideyukizushi/aes2-400-20240419134941
    ```
    # CV Result
    [Fold0] 0.8358919189593519
    [Fold1] 0.8363556728124214
    [Fold2] 0.8180511976390897
    [Fold3] 0.8332431669531231
    [Fold4] 0.8331952936995911
    ```


* **2024/04/29 add these great kernel ideas**
    * NewFE:count_spelling_errors
    * newFE:remove_punctuation
    * https://www.kaggle.com/code/xianhellg/more-feature-engineering-feature-selection-0-817https://www.kaggle.com/code/xianhellg/more-feature-engineering-feature-selection-0-817
    ```
    # Appendix.
    
    As a change from the original.
    In my experiments, I extract features with particularly high importance.
    /kaggle/input/aes2-400-fes-202404291649/usefe_list.pkl
    
    ```

---

In [ ]:
import gc
import torch
import copy
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments,DataCollatorWithPadding
import nltk
from datasets import Dataset
from glob import glob
import numpy as np 
import pandas as pd
import polars as pl
import re
import random
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from scipy.special import softmax
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier,GradientBoostingClassifier,BaggingClassifier
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB,MultinomialNB,ComplementNB
from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.pipeline import Pipeline
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score
from sklearn.metrics import cohen_kappa_score
from lightgbm import log_evaluation, early_stopping
import lightgbm as lgb
# nltk.download('wordnet')

In [ ]:
# add new function
add_model=0
six_features=0
threshold=0
gkf=0
gkf_smooth=0
gkf_as_feature=1

In [ ]:
MAX_LENGTH = 1024
TEST_DATA_PATH = "/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv"
if add_model:
    MODEL_PATH = [
        '/kaggle/input/aes2-400-20240419134941/*/*',
        '/kaggle/input/deberta-kaggle-0-815932/output/*/*'
    ]
else:
    MODEL_PATH = '/kaggle/input/aes2-400-20240419134941/*/*'
EVAL_BATCH_SIZE = 1

In [ ]:
if add_model:
    models = []
    for path in MODEL_PATH:
        models.extend(glob(path))
else:
    models = glob(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(models[0])

def tokenize(sample):
    return tokenizer(sample['full_text'], max_length=MAX_LENGTH, truncation=True)

df_test = pd.read_csv(TEST_DATA_PATH)
ds = Dataset.from_pandas(df_test).map(tokenize).remove_columns(['essay_id', 'full_text'])

args = TrainingArguments(
    ".", 
    per_device_eval_batch_size=EVAL_BATCH_SIZE, 
    report_to="none"
)

predictions = []
for model in models:
    model = AutoModelForSequenceClassification.from_pretrained(model)
    trainer = Trainer(
        model=model, 
        args=args, 
        data_collator=DataCollatorWithPadding(tokenizer), 
        tokenizer=tokenizer
    )    
    preds = trainer.predict(ds).predictions
    predictions.append(softmax(preds, axis=-1))
    del model, trainer
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
predicted_score = 0.
for p in predictions:
    predicted_score += p
    
predicted_score /= len(predictions)

In [ ]:
df_test['score'] = predicted_score.argmax(-1) + 1
df_test.head()

# **Data Loading**

In [ ]:
columns = [  
    (
        pl.col("full_text").str.split(by="\n\n").alias("paragraph")
    ),
]
PATH = "/kaggle/input/learning-agency-lab-automated-essay-scoring-2/"

train = pl.read_csv(PATH + "train.csv").with_columns(columns)
test = pl.read_csv(PATH + "test.csv").with_columns(columns)

train.head(1)

# **Preprocessing**

In [ ]:
cList = {
  "ain't": "am not","aren't": "are not","can't": "cannot","can't've": "cannot have","'cause": "because",  "could've": "could have","couldn't": "could not","couldn't've": "could not have","didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have","hasn't": "has not",
  "haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will","he'll've": "he will have","he's": "he is",
  "how'd": "how did","how'd'y": "how do you","how'll": "how will","how's": "how is","I'd": "I would","I'd've": "I would have","I'll": "I will","I'll've": "I will have","I'm": "I am","I've": "I have",
  "isn't": "is not","it'd": "it had","it'd've": "it would have","it'll": "it will", "it'll've": "it will have","it's": "it is","let's": "let us","ma'am": "madam","mayn't": "may not",
  "might've": "might have","mightn't": "might not","mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have","needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not","oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
  "shan't've": "shall not have","she'd": "she would","she'd've": "she would have","she'll": "she will","she'll've": "she will have","she's": "she is",
  "should've": "should have","shouldn't": "should not","shouldn't've": "should not have","so've": "so have","so's": "so is","that'd": "that would","that'd've": "that would have","that's": "that is","there'd": "there had","there'd've": "there would have","there's": "there is","they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have","they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we had",
  "we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
  "weren't": "were not","what'll": "what will","what'll've": "what will have",
  "what're": "what are","what's": "what is","what've": "what have","when's": "when is","when've": "when have",
  "where'd": "where did","where's": "where is","where've": "where have","who'll": "who will","who'll've": "who will have","who's": "who is","who've": "who have","why's": "why is",
  "why've": "why have","will've": "will have","won't": "will not","won't've": "will not have","would've": "would have","wouldn't": "would not",
  "wouldn't've": "would not have","y'all": "you all","y'alls": "you alls","y'all'd": "you all would",
  "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you had","you'd've": "you would have","you'll": "you you will","you'll've": "you you will have","you're": "you are",  "you've": "you have"
   }

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)
def dataPreprocessing(x):
    x = x.lower()
    x = removeHTML(x)
    x = re.sub("@\w+", '',x)
    x = re.sub("'\d+", '',x)
    x = re.sub("\d+", '',x)
    x = re.sub("http\w+", '',x)
    x = re.sub(r"\s+", " ", x)
#     x = expandContractions(x)
    x = re.sub(r"\.+", ".", x)
    x = re.sub(r"\,+", ",", x)
    x = x.strip()
    return x

# **Feature Engineering**

In [ ]:
import spacy
import re

nlp = spacy.load("en_core_web_sm")

with open('/kaggle/input/english-word-hx/words.txt', 'r') as file:
    english_vocab = set(word.strip().lower() for word in file)
    
def count_spelling_errors(text):
    doc = nlp(text)
    lemmatized_tokens = [token.lemma_.lower() for token in doc]
    spelling_errors = sum(1 for token in lemmatized_tokens if token not in english_vocab)
    return spelling_errors

In [ ]:
import string
def remove_punctuation(text):
    """
    Remove all punctuation from the input text.
    
    Args:
    - text (str): The input text.
    
    Returns:
    - str: The text with punctuation removed.
    """

    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

### 1.Paragraph Features

In [ ]:
def Paragraph_Preprocess(tmp):

    tmp = tmp.explode('paragraph')
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(dataPreprocessing))
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(remove_punctuation).alias('paragraph_no_pinctuation'))
    tmp = tmp.with_columns(pl.col('paragraph_no_pinctuation').map_elements(count_spelling_errors).alias("paragraph_error_num"))
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(lambda x: len(x)).alias("paragraph_len"))
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(lambda x: len(x.split('.'))).alias("paragraph_sentence_cnt"),
                    pl.col('paragraph').map_elements(lambda x: len(x.split(' '))).alias("paragraph_word_cnt"),)
    return tmp

# feature_eng
paragraph_fea = ['paragraph_len','paragraph_sentence_cnt','paragraph_word_cnt']
paragraph_fea2 = ['paragraph_error_num'] + paragraph_fea
def Paragraph_Eng(train_tmp):
    num_list = [0, 50,75,100,125,150,175,200,250,300,350,400,500,600]
    num_list2 = [0, 50,75,100,125,150,175,200,250,300,350,400,500,600,700]
    aggs = [
        *[pl.col('paragraph').filter(pl.col('paragraph_len') >= i).count().alias(f"paragraph_>{i}_cnt") for i in [0, 50,75,100,125,150,175,200,250,300,350,400,500,600,700] ], 
        *[pl.col('paragraph').filter(pl.col('paragraph_len') <= i).count().alias(f"paragraph_<{i}_cnt") for i in [25,49]], 
        *[pl.col(fea).max().alias(f"{fea}_max") for fea in paragraph_fea2],
        *[pl.col(fea).mean().alias(f"{fea}_mean") for fea in paragraph_fea2],
        *[pl.col(fea).min().alias(f"{fea}_min") for fea in paragraph_fea2],
        *[pl.col(fea).sum().alias(f"{fea}_sum") for fea in paragraph_fea2],
        *[pl.col(fea).first().alias(f"{fea}_first") for fea in paragraph_fea2],
        *[pl.col(fea).last().alias(f"{fea}_last") for fea in paragraph_fea2],
        *[pl.col(fea).kurtosis().alias(f"{fea}_kurtosis") for fea in paragraph_fea2],
        *[pl.col(fea).quantile(0.25).alias(f"{fea}_q1") for fea in paragraph_fea2],
        *[pl.col(fea).quantile(0.75).alias(f"{fea}_q3") for fea in paragraph_fea2],
        ]
    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas()
    return df
tmp = Paragraph_Preprocess(train)
train_feats = Paragraph_Eng(tmp)
train_feats['score'] = train['score']

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

### 2.Sentence Features

In [ ]:
def Sentence_Preprocess(tmp):
    tmp = tmp.with_columns(pl.col('full_text').map_elements(dataPreprocessing).str.split(by=".").alias("sentence"))
    tmp = tmp.explode('sentence')
    tmp = tmp.with_columns(pl.col('sentence').map_elements(lambda x: len(x)).alias("sentence_len"))
    tmp = tmp.with_columns(pl.col('sentence').map_elements(lambda x: len(x.split(' '))).alias("sentence_word_cnt"))    
    return tmp

# feature_eng
sentence_fea = ['sentence_len','sentence_word_cnt']
def Sentence_Eng(train_tmp):
    aggs = [
        *[pl.col('sentence').filter(pl.col('sentence_len') >= i).count().alias(f"sentence_>{i}_cnt") for i in [0,15,50,100,150,200,250,300] ], 
        *[pl.col('sentence').filter(pl.col('sentence_len') <= i).count().alias(f"sentence_<{i}_cnt") for i in [15,50] ], 
        *[pl.col(fea).max().alias(f"{fea}_max") for fea in sentence_fea],
        *[pl.col(fea).mean().alias(f"{fea}_mean") for fea in sentence_fea],
        *[pl.col(fea).min().alias(f"{fea}_min") for fea in sentence_fea],
        *[pl.col(fea).sum().alias(f"{fea}_sum") for fea in sentence_fea],
        *[pl.col(fea).first().alias(f"{fea}_first") for fea in sentence_fea],
        *[pl.col(fea).last().alias(f"{fea}_last") for fea in sentence_fea],
        *[pl.col(fea).kurtosis().alias(f"{fea}_kurtosis") for fea in sentence_fea],
        *[pl.col(fea).quantile(0.25).alias(f"{fea}_q1") for fea in sentence_fea],
        *[pl.col(fea).quantile(0.75).alias(f"{fea}_q3") for fea in sentence_fea],
    
        ]
    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas()
    return df

tmp = Sentence_Preprocess(train)
train_feats = train_feats.merge(Sentence_Eng(tmp), on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

### 3.Word Features

In [ ]:
# word feature
def Word_Preprocess(tmp):
    tmp = tmp.with_columns(pl.col('full_text').map_elements(dataPreprocessing).str.split(by=" ").alias("word"))
    tmp = tmp.explode('word')
    tmp = tmp.with_columns(pl.col('word').map_elements(lambda x: len(x)).alias("word_len"))
    tmp = tmp.filter(pl.col('word_len')!=0)    
    return tmp

# feature_eng
def Word_Eng(train_tmp):
    aggs = [
        *[pl.col('word').filter(pl.col('word_len') >= i+1).count().alias(f"word_{i+1}_cnt") for i in range(15) ], 
        pl.col('word_len').max().alias(f"word_len_max"),
        pl.col('word_len').mean().alias(f"word_len_mean"),
        pl.col('word_len').std().alias(f"word_len_std"),
        pl.col('word_len').quantile(0.25).alias(f"word_len_q1"),
        pl.col('word_len').quantile(0.50).alias(f"word_len_q2"),
        pl.col('word_len').quantile(0.75).alias(f"word_len_q3"),
        ]
    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas()
    return df

tmp = Word_Preprocess(train)
train_feats = train_feats.merge(Word_Eng(tmp), on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

### 4.Tf-idf features

In [ ]:
vectorizer = TfidfVectorizer(
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(3,6),
            min_df=0.05,
            max_df=0.95,
            sublinear_tf=True,
)

train_tfid = vectorizer.fit_transform([i for i in train['full_text']])
dense_matrix = train_tfid.toarray()
df = pd.DataFrame(dense_matrix)
tfid_columns = [ f'tfid_{i}' for i in range(len(df.columns))]
df.columns = tfid_columns
df['essay_id'] = train_feats['essay_id']
train_feats = train_feats.merge(df, on='essay_id', how='left')
feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Number of Features: ',len(feature_names))
train_feats.head(3)

### 5.CountVectorizer Features

In [ ]:
vectorizer_cnt = CountVectorizer(
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(2,3),
            min_df=0.10,
            max_df=0.85,
)
train_tfid = vectorizer_cnt.fit_transform([i for i in train['full_text']])
dense_matrix = train_tfid.toarray()
df = pd.DataFrame(dense_matrix)
tfid_columns = [ f'tfid_cnt_{i}' for i in range(len(df.columns))]
df.columns = tfid_columns
df['essay_id'] = train_feats['essay_id']
train_feats = train_feats.merge(df, on='essay_id', how='left')

### 6.Six-features

In [ ]:
if six_features:
    !pip install --no-index --no-deps /kaggle/input/aes-whls/aes_whls/sentence_transformers-2.8.0.dev0-py3-none-any.whl
    train_f = pd.read_csv(PATH + "train.csv")
    test_f = pd.read_csv(PATH + "test.csv")
    from sentence_transformers import SentenceTransformer, models

    feedback_df = pd.read_csv('/kaggle/input/feedback-data/feedback_data.csv')

    feed_embeds = []

    merged_embeds = []

    test_embeds = []

    for i in range(5):
        model_path = f'/kaggle/input/sent-debsmall/deberta_small_trained/temp_fold{i}_checkpoints'
        word_embedding_model = models.Transformer(model_path, max_seq_length=1024)
        pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
        model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

        model.half()
        model = model.to('cuda')

        feed_custom_embeddings_train = model.encode(feedback_df.loc[:, 'full_text'].values, device='cuda',
                                                    show_progress_bar=True, normalize_embeddings=True)

        feed_embeds.append(feed_custom_embeddings_train)

        merged_custom_embeddings = model.encode(train_f.loc[:, 'full_text'].values, device='cuda',
                                                show_progress_bar=True, normalize_embeddings=True)

        merged_embeds.append(merged_custom_embeddings)


        test_custom_embeddings = model.encode(test_f.loc[:, 'full_text'].values, device='cuda',
                                                show_progress_bar=True, normalize_embeddings=True)

        test_embeds.append(test_custom_embeddings)

    feed_embeds = np.mean(feed_embeds, axis=0)
    merged_embeds = np.mean(merged_embeds, axis=0)
    test_embeds = np.mean(test_embeds, axis=0)

    from sklearn.linear_model import Ridge
    from sklearn.multioutput import MultiOutputRegressor

    targets = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']


    ridge = Ridge(alpha=1.0)

    multioutputregressor = MultiOutputRegressor(ridge)



    multioutputregressor.fit(feed_embeds, feedback_df.loc[:, targets])

    feedback_predictions = multioutputregressor.predict(merged_embeds)

    feedback_predictions_df = pd.DataFrame(feedback_predictions, columns=targets)

    test_feedback_predictions = multioutputregressor.predict(test_embeds)

    test_feedback_predictions_df = pd.DataFrame(test_feedback_predictions, columns=targets)

    print(feedback_predictions_df)# .head()

    train_feats = pd.concat((train_feats, feedback_predictions_df), axis=1)
    
train_feats.shape

### 7.Meta Features(deberta-v3-large)

In [ ]:
import joblib

deberta_oof = joblib.load('/kaggle/input/aes2-400-20240419134941/oof.pkl')
print(deberta_oof.shape, train_feats.shape)

for i in range(6):
    train_feats[f'deberta_oof_{i}'] = deberta_oof[:, i]

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))    

train_feats.shape

### 7.prompt Features

In [ ]:
gkf_train_df = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv")
persuade = pd.read_csv("/kaggle/input/persaude-corpus-2/persuade_2.0_human_scores_demo_id_github.csv")

intersection = pd.merge(gkf_train_df, persuade, on="full_text", how="inner")[["essay_id", "full_text", "prompt_name"]].reset_index(drop=True)
difference = gkf_train_df[~gkf_train_df["essay_id"].isin(intersection["essay_id"])].reset_index(drop=True)

intersection["label"] =  intersection["prompt_name"].astype("category").cat.codes
intersection.head()

prompt_to_id = intersection.drop_duplicates(subset=("prompt_name", "label"))[["prompt_name", "label"]]
label2id = {row["prompt_name"]: row["label"] for _, row in prompt_to_id.iterrows()}
id2label = {row["label"]: row["prompt_name"] for _, row in prompt_to_id.iterrows()}


def get_prompt(gkf_df,df,tokenizer):
    
    intersection = pd.merge(gkf_df, persuade, on="full_text", how="inner")[["essay_id", "full_text", "prompt_name"]].reset_index(drop=True)
    difference = gkf_df[~gkf_df["essay_id"].isin(intersection["essay_id"])].reset_index(drop=True)
    print("len(intersection):", len(intersection))
    print("len(difference):", len(difference))
    
    
    intersection_ds = Dataset.from_pandas(intersection)
    diff_ds = Dataset.from_pandas(difference)
    intersection_ds = intersection_ds.map(lambda i: tokenizer(i["full_text"], max_length=1024, truncation=True), batched=True)
    diff_ds = diff_ds.map(lambda i: tokenizer(i["full_text"], max_length=1024, truncation=True), batched=True)
    
    MODEL_PATHS_PROMPT = [
        '/kaggle/input/aes-train-prompt-added-csv/output/*/*'
    ]
    EVAL_BATCH_SIZE = 1

    model_pro = []
    for path in MODEL_PATHS_PROMPT:
        model_pro.extend(glob(path))

    tokenizer = AutoTokenizer.from_pretrained(model_pro[0])

    def tokenize(sample):
        return tokenizer(sample['full_text'], max_length=MAX_LENGTH, truncation=True)

    df_test = pd.read_csv(TEST_DATA_PATH)
    ds = Dataset.from_pandas(df_test).map(tokenize).remove_columns(['essay_id', 'full_text'])

    args = TrainingArguments(
        ".", 
        per_device_eval_batch_size=EVAL_BATCH_SIZE, 
        report_to="none"
    )
    
    if len(difference)!=0:
        predictions = []
        for model in model_pro:
            model = AutoModelForSequenceClassification.from_pretrained(model)
            trainer = Trainer(
                model=model, 
                args=args, 
                data_collator=DataCollatorWithPadding(tokenizer), 
                tokenizer=tokenizer
            )    
            preds = trainer.predict(diff_ds).predictions
            predictions.append(softmax(preds, axis=-1))
            del model, trainer
            torch.cuda.empty_cache()
            gc.collect()

        predicted_score = 0.
        for p in predictions:
            predicted_score += p

        predicted_score /= len(predictions)

        
    if len(difference)!=0:
        predictions=np.array(predicted_score)
        predicted_score=dict()
        for i in range(7):
            predicted_score[f'prompt_{i}'] = predictions[:, i]

        essay_id = diff_ds["essay_id"]
        result_df = pd.DataFrame({"essay_id": essay_id})

        for i in range(7):
            result_df[f"prompt_{i}"] = predicted_score[f"prompt_{i}"]


    if len(intersection)!=0:
        prompt_id = [label2id[i] for i in intersection["prompt_name"]]
        intersection['prompt_id']=prompt_id

        intersection_df = intersection[["essay_id"]].copy()
        for i in range(7):
            prompt_i=[1 if abs(i-j)<0.01 else 0 for j in intersection['prompt_id']]
            intersection_df[f"prompt_{i}"]=prompt_i



    if len(difference)!=0 and len(intersection)!=0:
        final_df = pd.concat([result_df, intersection_df])
    elif len(difference)!=0 and len(intersection)==0:
        final_df=result_df
    else:
        final_df=intersection_df

    prompts_df=pd.merge(df,final_df,on="essay_id")

    return prompts_df


if gkf_as_feature:
    gkf_train_df = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv")
    gkf_test_df = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv")
    df_train=pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv")
    train_prompts_df=get_prompt(gkf_train_df,df_train,tokenizer)
    print(train_prompts_df.head(3))
    test_prompts_df=get_prompt(gkf_test_df,df_test,tokenizer)
    print(test_prompts_df.head(3))

In [ ]:
if gkf_as_feature:
    for i in range(7):
        train_feats[f'prompt_{i}'] = train_prompts_df[f'prompt_{i}']

train_feats.shape

# **Model training**

In [ ]:
def quadratic_weighted_kappa(y_true, y_pred):
    y_true = y_true + a
    y_pred = (y_pred + a).clip(1, 6).round()
    qwk = cohen_kappa_score(y_true, y_pred, weights="quadratic")
    return 'QWK', qwk, True
def qwk_obj(y_true, y_pred):
    labels = y_true + a
    preds = y_pred + a
    preds = preds.clip(1, 6)
    f = 1/2*np.sum((preds-labels)**2)
    g = 1/2*np.sum((preds-a)**2+b)
    df = preds - labels
    dg = preds - a
    grad = (df/g - f*dg/g**2)*len(labels)
    hess = np.ones(len(labels))
    return grad, hess
a = 2.998
b = 1.092

# **FeatureSelection(Load High importance FE)**

In [ ]:
import pickle
with open('/kaggle/input/aes2-400-fes-202404291649/usefe_list.pkl', mode='br') as fi:
  feature_names = pickle.load(fi)
if six_features:
    feature_select = feature_names
    six_features=['cohesion','syntax','vocabulary','phraseology','grammar','conventions']  # add
    feature_select=feature_select+six_features
elif gkf_as_feature:
    feature_select = feature_names
    prompt_features=['prompt_0','prompt_1','prompt_2','prompt_3','prompt_4','prompt_5','prompt_6']  # add
    feature_select=feature_select+prompt_features
else:
    feature_select = feature_names

In [ ]:
X = train_feats[feature_select].astype(np.float32).values

y_split = train_feats['score'].astype(int).values
y = train_feats['score'].astype(np.float32).values-a
if threshold:
    oof = np.zeros(len(train), dtype='float32')
    x_score_thres = train_feats['score'].astype(int).values  # add
else:
    oof = train_feats['score'].astype(int).values


In [ ]:
len(feature_select)

# **Let's use cross-validation**

In [ ]:
if gkf:
    n_splits = 7 
    
    from sklearn.model_selection import GroupKFold
    gkf=GroupKFold(n_splits=7)
    prompts_df=pd.read_csv("/kaggle/input/prompt-df/prompt_df.csv")
    preds_valid1=np.zeros(len(prompts_df))
    
    f1_scores = []
    kappa_scores = []
    models = []
    predictions = []
    callbacks = [log_evaluation(period=64), early_stopping(stopping_rounds=256,first_metric_only=True)]
    
    i=1
    for train_index, test_index in gkf.split(prompts_df,prompts_df["score"],groups=prompts_df["prompt_name"]):

        print('fold',i)
        X_train_fold, X_test_fold = X[train_index], X[test_index]


        y_train_fold, y_test_fold, y_test_fold_int = y[train_index], y[test_index], y_split[test_index]

        model = lgb.LGBMRegressor(
                    objective = qwk_obj,
                    metrics = 'None',
                    learning_rate = 0.1,     # 0.05
                    max_depth = 8,           # 5
                    num_leaves = 10,         # 10
                    colsample_bytree=0.5,    # 0.3
                    reg_alpha = 0.1,         # 0.7
                    reg_lambda = 0.8,        # 0.1
                    n_estimators=1024,        # 700
                    random_state=42,         # 42
                    extra_trees=True,
                    class_weight='balanced',
                    verbosity = - 1)
    
        predictor = model.fit(X_train_fold,
                                      y_train_fold,
                                      eval_names=['train', 'valid'],
                                      eval_set=[(X_train_fold, y_train_fold), (X_test_fold, y_test_fold)],
                                      eval_metric=quadratic_weighted_kappa,
                                      callbacks=callbacks,)
        models.append(predictor)
        predictions_fold = predictor.predict(X_test_fold)
        predictions_fold = predictions_fold + a
        oof[test_index]=predictions_fold
        predictions_fold = predictions_fold.clip(1, 6).round()
        predictions.append(predictions_fold)
        f1_fold = f1_score(y_test_fold_int, predictions_fold, average='weighted')
        f1_scores.append(f1_fold)


        kappa_fold = cohen_kappa_score(y_test_fold_int, predictions_fold, weights='quadratic')
        kappa_scores.append(kappa_fold)

        cm = confusion_matrix(y_test_fold_int, predictions_fold, labels=[x for x in range(1,7)])
    
        disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                      display_labels=[x for x in range(1,7)])
        disp.plot()
        plt.show()
        print(f'F1 score across fold: {f1_fold}')
        print(f'Cohen kappa score across fold: {kappa_fold}')
        i+=1
    
    mean_f1_score = np.mean(f1_scores)
    mean_kappa_score = np.mean(kappa_scores)
else:
    n_splits = 15

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)

    f1_scores = []
    kappa_scores = []
    models = []
    predictions = []
    callbacks = [log_evaluation(period=25), early_stopping(stopping_rounds=75,first_metric_only=True)]

    i=1
    for train_index, test_index in skf.split(X, y_split):

        print('fold',i)
        X_train_fold, X_test_fold = X[train_index], X[test_index]


        y_train_fold, y_test_fold, y_test_fold_int = y[train_index], y[test_index], y_split[test_index]

        model = lgb.LGBMRegressor(
                    objective = qwk_obj,
                    metrics = 'None',
                    learning_rate = 0.05,
                    max_depth = 5,
                    num_leaves = 10,
                    colsample_bytree=0.3,
                    reg_alpha = 0.7,
                    reg_lambda = 0.1,
                    n_estimators=700,
                    random_state=42,
                    extra_trees=True,
                    class_weight='balanced',
                    verbosity = - 1)

        predictor = model.fit(X_train_fold,
                                      y_train_fold,
                                      eval_names=['train', 'valid'],
                                      eval_set=[(X_train_fold, y_train_fold), (X_test_fold, y_test_fold)],
                                      eval_metric=quadratic_weighted_kappa,
                                      callbacks=callbacks,)
        models.append(predictor)
        predictions_fold = predictor.predict(X_test_fold)
        predictions_fold = predictions_fold + a
        oof[test_index]=predictions_fold
        predictions_fold = predictions_fold.clip(1, 6).round()
        predictions.append(predictions_fold)
        f1_fold = f1_score(y_test_fold_int, predictions_fold, average='weighted')
        f1_scores.append(f1_fold)


        kappa_fold = cohen_kappa_score(y_test_fold_int, predictions_fold, weights='quadratic')
        kappa_scores.append(kappa_fold)

        cm = confusion_matrix(y_test_fold_int, predictions_fold, labels=[x for x in range(1,7)])

        disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                      display_labels=[x for x in range(1,7)])
        disp.plot()
        plt.show()
        print(f'F1 score across fold: {f1_fold}')
        print(f'Cohen kappa score across fold: {kappa_fold}')
        i+=1

In [ ]:
mean_f1_score = np.mean(f1_scores)
mean_kappa_score = np.mean(kappa_scores)

print("="*50)
print(f'Mean F1 score across {n_splits} folds: {mean_f1_score}')
print(f'Mean Cohen kappa score across {n_splits} folds: {mean_kappa_score}')
print("="*50)

In [ ]:
oof

# **threshold**

In [ ]:
def find_thresholds(true, pred, steps=50):

    # SAVE TRIALS FOR PLOTTING
    xs = [[],[],[],[],[]]
    ys = [[],[],[],[],[]]

    # COMPUTE BASELINE METRIC
    threshold = [1.5, 2.5, 3.5, 4.5, 5.5]
    pred2 = pd.cut(pred, [-np.inf] + threshold + [np.inf], 
                    labels=[1,2,3,4,5,6]).astype('int32')
    best = cohen_kappa_score(true, pred2, weights="quadratic")
    # cohen_kappa_score(y_test_fold_int, predictions_fold, weights='quadratic')

    # FIND FIVE OPTIMAL THRESHOLDS
    for k in range(5):
        for sign in [1,-1]:
            v = threshold[k]
            threshold2 = threshold.copy()
            stop = 0
            while stop<steps:

                # TRY NEW THRESHOLD
                v += sign * 0.001
                threshold2[k] = v
                pred2 = pd.cut(pred, [-np.inf] + threshold2 + [np.inf], 
                                labels=[1,2,3,4,5,6]).astype('int32')
                metric = cohen_kappa_score(true, pred2, weights="quadratic")

                # SAVE TRIALS FOR PLOTTING
                xs[k].append(v)
                ys[k].append(metric)

                # EARLY STOPPING
                if metric<=best:
                    stop += 1
                else:
                    stop = 0
                    best = metric
                    threshold = threshold2.copy()

    # COMPUTE FINAL METRIC
    pred2 = pd.cut(pred, [-np.inf] + threshold + [np.inf], 
                    labels=[1,2,3,4,5,6]).astype('int32')
    best = cohen_kappa_score(true, pred2, weights="quadratic")   

    # RETURN RESULTS
    threshold = [np.round(t,3) for t in threshold]
    return best, threshold, xs, ys

if threshold:
    best, thresholds, xs, ys = find_thresholds(x_score_thres, oof, steps=500)
    print('Best thresholds are:', thresholds )
    print('=> achieve Overall CV QWK score =', best )

In [ ]:
# import pickle

# with open('models.pkl', 'wb') as f:
#     pickle.dump(models, f)

In [ ]:
# with open('models.pkl', 'rb') as f:
#     models = pickle.load(f)

# **Inference**

In [ ]:
# Paragraph
tmp = Paragraph_Preprocess(test)
test_feats = Paragraph_Eng(tmp)
# Sentence
tmp = Sentence_Preprocess(test)
test_feats = test_feats.merge(Sentence_Eng(tmp), on='essay_id', how='left')
# Word
tmp = Word_Preprocess(test)
test_feats = test_feats.merge(Word_Eng(tmp), on='essay_id', how='left')

# Tfidf
test_tfid = vectorizer.transform([i for i in test['full_text']])
dense_matrix = test_tfid.toarray()
df = pd.DataFrame(dense_matrix)
tfid_columns = [ f'tfid_{i}' for i in range(len(df.columns))]
df.columns = tfid_columns
df['essay_id'] = test_feats['essay_id']
test_feats = test_feats.merge(df, on='essay_id', how='left')

# CountVectorizer
test_tfid = vectorizer_cnt.transform([i for i in test['full_text']])
dense_matrix = test_tfid.toarray()
df = pd.DataFrame(dense_matrix)
tfid_columns = [ f'tfid_cnt_{i}' for i in range(len(df.columns))]
df.columns = tfid_columns
df['essay_id'] = test_feats['essay_id']
test_feats = test_feats.merge(df, on='essay_id', how='left')

if six_features:
    test_feats = pd.concat((test_feats, test_feedback_predictions_df), axis=1)
if gkf_as_feature:
    for i in range(7):
        test_feats[f'prompt_{i}'] = test_prompts_df[f'prompt_{i}']

for i in range(6):
    test_feats[f'deberta_oof_{i}'] = predicted_score[:, i]

# Features number
feature_names = list(filter(lambda x: x not in ['essay_id','score'], test_feats.columns))
print('Features number: ',len(feature_names))
test_feats.head(3)

# **Gkf**

In [ ]:
import plotly.express as px
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, EvalPrediction
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay

In [ ]:
if gkf:
    gkf_train_df = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv")
    # gkf_test_df = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv")
    gkf_test_df = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv")
    persuade = pd.read_csv("/kaggle/input/persaude-corpus-2/persuade_2.0_human_scores_demo_id_github.csv")
    
    intersection = pd.merge(gkf_train_df, persuade, on="full_text", how="inner")[["essay_id", "full_text", "prompt_name"]].reset_index(drop=True)
    difference = gkf_train_df[~gkf_train_df["essay_id"].isin(intersection["essay_id"])].reset_index(drop=True)
    print("len(intersection):", len(intersection))
    print("len(difference):", len(difference))
    
    intersection["label"] =  intersection["prompt_name"].astype("category").cat.codes
    intersection.head()
    
    prompt_to_id = intersection.drop_duplicates(subset=("prompt_name", "label"))[["prompt_name", "label"]]
    label2id = {row["prompt_name"]: row["label"] for _, row in prompt_to_id.iterrows()}
    id2label = {row["label"]: row["prompt_name"] for _, row in prompt_to_id.iterrows()}
    prompt_to_id
    
    intersection = pd.merge(gkf_test_df, persuade, on="full_text", how="inner")[["essay_id", "full_text", "prompt_name"]].reset_index(drop=True)
    difference = gkf_test_df[~gkf_test_df["essay_id"].isin(intersection["essay_id"])].reset_index(drop=True)
    print("len(intersection):", len(intersection))
    print("len(difference):", len(difference))
    
    intersection_ds = Dataset.from_pandas(intersection)
    diff_ds = Dataset.from_pandas(difference)
    intersection_ds = intersection_ds.map(lambda i: tokenizer(i["full_text"], max_length=1024, truncation=True), batched=True)
    diff_ds = diff_ds.map(lambda i: tokenizer(i["full_text"], max_length=1024, truncation=True), batched=True)
    
    MODEL_PATHS_PROMPT = [
        '/kaggle/input/aes-train-prompt-added-csv/output/*/*'
    ]
    EVAL_BATCH_SIZE = 1

    model_pro = []
    for path in MODEL_PATHS_PROMPT:
        model_pro.extend(glob(path))

    tokenizer = AutoTokenizer.from_pretrained(model_pro[0])

    def tokenize(sample):
        return tokenizer(sample['full_text'], max_length=MAX_LENGTH, truncation=True)

    df_test = pd.read_csv(TEST_DATA_PATH)
    ds = Dataset.from_pandas(df_test).map(tokenize).remove_columns(['essay_id', 'full_text'])

    args = TrainingArguments(
        ".", 
        per_device_eval_batch_size=EVAL_BATCH_SIZE, 
        report_to="none"
    )
    
    if len(difference)!=0:
        predictions = []
        for model in model_pro:
            model = AutoModelForSequenceClassification.from_pretrained(model)
            trainer = Trainer(
                model=model, 
                args=args, 
                data_collator=DataCollatorWithPadding(tokenizer), 
                tokenizer=tokenizer
            )    
            preds = trainer.predict(diff_ds).predictions
            predictions.append(softmax(preds, axis=-1))
            del model, trainer
            torch.cuda.empty_cache()
            gc.collect()

        predicted_score = 0.
        for p in predictions:
            predicted_score += p

        predicted_score /= len(predictions)

        print("-------predicted_score--------:")
        print(predicted_score)
        
    if gkf_smooth:
        if len(difference)!=0:
            predictions=np.array(predicted_score)
            predicted_score=dict()
            for i in range(7):
                predicted_score[f'prompt_{i}'] = predictions[:, i]

            essay_id = diff_ds["essay_id"]
            result_df = pd.DataFrame({"essay_id": essay_id})

            for i in range(7):
                result_df[f"prompt_{i}"] = predicted_score[f"prompt_{i}"]
            print("-------result_df--------:")
            print(result_df)
        
        if len(intersection)!=0:
            prompt_id = [label2id[i] for i in intersection["prompt_name"]]
            intersection['prompt_id']=prompt_id

            intersection_df = intersection[["essay_id"]].copy()
            for i in range(7):
                if i==int(intersection['prompt_id']):
                    intersection_df[f"prompt_{i}"] = 1
                else:
                    intersection_df[f"prompt_{i}"] = 0
            print("-------intersection_df--------:")
            print(intersection_df)
        
        if len(difference)!=0 and len(intersection)!=0:
            final_df = pd.concat([result_df, intersection_df])
        elif len(difference)!=0 and len(intersection)==0:
            final_df=result_df
        else:
            final_df=intersection_df
        print("-------final_df--------:")
        print(final_df)
        prompts_df=pd.merge(df_test,final_df,on="essay_id")
        print("-------prompts_df--------:")
        print(prompts_df)
    else:
        predicted_score = 0.
        for p in predictions:
            predicted_score += p

        predicted_score /= len(predictions)
        predicted_score
        y_pred=predicted_score.argmax(-1)
        y_pred

        essay_id = diff_ds["essay_id"]
        prompt_name = [id2label[i] for i in predicted_score.argmax(-1)]  # convert prompt id back to prompt name
        result_df = pd.DataFrame({"essay_id": essay_id, "prompt_name": prompt_name, "predicted": [True] * len(essay_id)})
        result_df
        intersection_df = intersection[["essay_id", "prompt_name"]].copy()
        intersection_df["predicted"] = False
        final_df = pd.concat([result_df, intersection_df])
        final_df.to_csv("predicted_prompt.csv", index=False)
        final_df
        prompts_df=pd.merge(df_test,final_df,on="essay_id")
        prompts_df
        prompt_name = [label2id[i] for i in prompts_df["prompt_name"]]
        prompts_df['prompt_id']=prompt_name
        print("-------prompts_df--------:")
        print(prompts_df)

In [ ]:
probabilities = []
for model in models:
    print(model)
    proba= model.predict(test_feats[feature_select])+ a
    probabilities.append(proba)

if gkf_smooth:
    prompt_Pro = pd.DataFrame({f"prompt_{0}": prompts_df[f"prompt_{0}"]})
    for i in range(1,7):
        prompt_Pro[f"prompt_{i}"] = prompts_df[f"prompt_{i}"]
    prompt_Pro=np.array(prompt_Pro)
    probabilities=np.array(probabilities)
    probabilities=probabilities.T
    predictions=[]
    
    for i in range(len(prompt_Pro)):
        predictions.append(np.dot(prompt_Pro[i],probabilities[i]))
    predictions=np.array(predictions)
    print(predictions)
else:
    predictions = np.mean(probabilities, axis=0)
    print(predictions)


if threshold:
    predictions = pd.cut(predictions, [-np.inf] + thresholds + [np.inf], 
                       labels=[1,2,3,4,5,6]).astype('int32')  # add
else:
    predictions = np.round(predictions.clip(1, 6))

print(predictions)

In [ ]:
submission=pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv")
submission['score']=predictions
submission['score']=submission['score'].astype(int)
submission.to_csv("submission.csv",index=None)
display(submission.head())